 NAME, and clearly specify the inputs/outputs of your program. It should specify the appropriate command (and input parameters if relevant)
to compile (and/or execute) your program.<br>
Artificial intelligence CSC4444 Project 1 <br>
Author: Jake Ardoin<br>
Date: 9/27/09<br>

In order to run the program, you have to create an instance of Vaccuum_World puzzle with the parameters:
1. initial state
2. (optional) goal state (default state is set to (0,0,0,0,0,0,0,0,0), which is when all squares are clean

The states are a 2-tuple containing a 2-tuple for vaccuum location and a 9-tuple for the dirt distribution.
example_state = ((1,1), (0,0,1,1,0,1,1,0,0))

Next you pick a search function (here either best_search or astar) and input the puzzle as a parameter to it, along with
an optinal heuristic function. if you don't give it a heruistic function, it will use the defaul one defined
inside Vaccuum_World.

Like this: astar_search(puzzle, h2)

That will return the node which is the solution. calling the method soltuion() on it will give you the solution. 

Like this: astar_search(puzzle, h2).solution()

Other commands that can be called are defined inside the Node class, since the return value
of the graph functions are Nodes.

The first 2 cells of this notebook are copies of code that I got from the textbook's github. They
define some basic problem and node classes and the graphing functions. 

The third cell defines my Vaccuum_World class.
The fourth cell is parameters I've defined for my examples.

The last 2 cells are my examples, the first using the defaul heruistic, h1, the second passing the h2
parameter to the graphing function to test a different heuristic.

So h1 can be found inside the Vaccuum_World class, h2 can be found defined where I defined the parameters for
my examples.

This is all noted in comments in the cells. 


This is the problem that I will be simulating and solving with a custom made vaccuum_world class<br><br>
Consider the vacuum-cleaning problem outlined below:<br>
Here we have a vacuum-cleaning agent that can sense the environment and perform actions to move around and vacuum-clean dirty squares.<br><br>
We make the following assumptions:<br>
• We assume a 3 by 3 grid world known to the agent. The environment is fully observable: the percepts give complete information about the Dirty/Clean status of every square and the agent’s location.<br>
• The environment is deterministic: A clean square remains clean and a dirty square remains dirty unless the agent cleans it up.<br>
• The actions available for the agent are: Left, Right, Up, Down, Suck. Each action takes place in one time "step".<br>
• The actions and percepts are perfectly reliable.<br>
• Each of the actions will incur a penalty of −1 point.<br>
• The agent will get a penalty of -2 points per dirty square per time step,<br>
• The agent’s performance is measured by the sum of negative points received on reaching a state with all squares in the environment being "Clean". As usual, a rational agent should maximize its performance score.

In [1]:
# this is the utils.py file found in the github https://github.com/aimacode/aima-python which powers some of the 
# classes below that are also from there.

"""Provides some utilities widely used by other modules"""

import bisect
import collections
import collections.abc
import heapq
import operator
import os.path
import random
import math
import functools
import numpy as np
from itertools import chain, combinations


# ______________________________________________________________________________
# Functions on Sequences and Iterables


def sequence(iterable):
    """Converts iterable to sequence, if it is not already one."""
    return (iterable if isinstance(iterable, collections.abc.Sequence)
            else tuple([iterable]))


def removeall(item, seq):
    """Return a copy of seq (or string) with all occurrences of item removed."""
    if isinstance(seq, str):
        return seq.replace(item, '')
    elif isinstance(seq, set):
        rest = seq.copy()
        rest.remove(item)
        return rest
    else:
        return [x for x in seq if x != item]


def unique(seq):
    """Remove duplicate elements from seq. Assumes hashable elements."""
    return list(set(seq))


def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))


def multimap(items):
    """Given (key, val) pairs, return {key: [val, ....], ...}."""
    result = collections.defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return dict(result)


def multimap_items(mmap):
    """Yield all (key, val) pairs stored in the multimap."""
    for (key, vals) in mmap.items():
        for val in vals:
            yield key, val


def product(numbers):
    """Return the product of the numbers, e.g. product([2, 3, 10]) == 60"""
    result = 1
    for x in numbers:
        result *= x
    return result


def first(iterable, default=None):
    """Return the first element of an iterable; or default."""
    return next(iter(iterable), default)


def is_in(elt, seq):
    """Similar to (elt in seq), but compares with 'is', not '=='."""
    return any(x is elt for x in seq)


def mode(data):
    """Return the most common data item. If there are ties, return any one of them."""
    [(item, count)] = collections.Counter(data).most_common(1)
    return item


def powerset(iterable):
    """powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"""
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s) + 1)))[1:]


def extend(s, var, val):
    """Copy dict s and extend it by setting var to val; return copy."""
    s2 = s.copy()
    s2[var] = val
    return s2


# ______________________________________________________________________________
# argmin and argmax

identity = lambda x: x

argmin = min
argmax = max


def argmin_random_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return argmin(shuffled(seq), key=key)


def argmax_random_tie(seq, key=identity):
    """Return an element with highest fn(seq[i]) score; break ties at random."""
    return argmax(shuffled(seq), key=key)


def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items


# ______________________________________________________________________________
# Statistical and mathematical functions


def histogram(values, mode=0, bin_function=None):
    """Return a list of (value, count) pairs, summarizing the input values.
    Sorted by increasing value, or if mode=1, by decreasing count.
    If bin_function is given, map it over values first."""
    if bin_function:
        values = map(bin_function, values)

    bins = {}
    for val in values:
        bins[val] = bins.get(val, 0) + 1

    if mode:
        return sorted(list(bins.items()), key=lambda x: (x[1], x[0]),
                      reverse=True)
    else:
        return sorted(bins.items())


def dotproduct(X, Y):
    """Return the sum of the element-wise product of vectors X and Y."""
    return sum(x * y for x, y in zip(X, Y))


def element_wise_product(X, Y):
    """Return vector as an element-wise product of vectors X and Y"""
    assert len(X) == len(Y)
    return [x * y for x, y in zip(X, Y)]


def matrix_multiplication(X_M, *Y_M):
    """Return a matrix as a matrix-multiplication of X_M and arbitrary number of matrices *Y_M"""

    def _mat_mult(X_M, Y_M):
        """Return a matrix as a matrix-multiplication of two matrices X_M and Y_M
        >>> matrix_multiplication([[1, 2, 3],
                                   [2, 3, 4]],
                                   [[3, 4],
                                    [1, 2],
                                    [1, 0]])
        [[8, 8],[13, 14]]
        """
        assert len(X_M[0]) == len(Y_M)

        result = [[0 for i in range(len(Y_M[0]))] for j in range(len(X_M))]
        for i in range(len(X_M)):
            for j in range(len(Y_M[0])):
                for k in range(len(Y_M)):
                    result[i][j] += X_M[i][k] * Y_M[k][j]
        return result

    result = X_M
    for Y in Y_M:
        result = _mat_mult(result, Y)

    return result


def vector_to_diagonal(v):
    """Converts a vector to a diagonal matrix with vector elements
    as the diagonal elements of the matrix"""
    diag_matrix = [[0 for i in range(len(v))] for j in range(len(v))]
    for i in range(len(v)):
        diag_matrix[i][i] = v[i]

    return diag_matrix


def vector_add(a, b):
    """Component-wise addition of two vectors."""
    return tuple(map(operator.add, a, b))


def scalar_vector_product(X, Y):
    """Return vector as a product of a scalar and a vector"""
    return [X * y for y in Y]


def scalar_matrix_product(X, Y):
    """Return matrix as a product of a scalar and a matrix"""
    return [scalar_vector_product(X, y) for y in Y]


def inverse_matrix(X):
    """Inverse a given square matrix of size 2x2"""
    assert len(X) == 2
    assert len(X[0]) == 2
    det = X[0][0] * X[1][1] - X[0][1] * X[1][0]
    assert det != 0
    inv_mat = scalar_matrix_product(1.0 / det, [[X[1][1], -X[0][1]], [-X[1][0], X[0][0]]])

    return inv_mat


def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)


def weighted_sample_with_replacement(n, seq, weights):
    """Pick n samples from seq at random, with replacement, with the
    probability of each element in proportion to its corresponding
    weight."""
    sample = weighted_sampler(seq, weights)

    return [sample() for _ in range(n)]


def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)

    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]


def weighted_choice(choices):
    """A weighted version of random.choice"""
    # NOTE: Shoule be replaced by random.choices if we port to Python 3.6

    total = sum(w for _, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w >= r:
            return c, w
        upto += w


def rounder(numbers, d=4):
    """Round a single number, or sequence of numbers, to d decimal places."""
    if isinstance(numbers, (int, float)):
        return round(numbers, d)
    else:
        constructor = type(numbers)  # Can be list, set, tuple, etc.
        return constructor(rounder(n, d) for n in numbers)


def num_or_str(x):  # TODO: rename as `atom`
    """The argument is a string; convert to a number if
       possible, or strip it."""
    try:
        return int(x)
    except ValueError:
        try:
            return float(x)
        except ValueError:
            return str(x).strip()


def normalize(dist):
    """Multiply each number by a constant such that the sum is 1.0"""
    if isinstance(dist, dict):
        total = sum(dist.values())
        for key in dist:
            dist[key] = dist[key] / total
            assert 0 <= dist[key] <= 1, "Probabilities must be between 0 and 1."
        return dist
    total = sum(dist)
    return [(n / total) for n in dist]


def norm(X, n=2):
    """Return the n-norm of vector X"""
    return sum([x ** n for x in X]) ** (1 / n)


def clip(x, lowest, highest):
    """Return x clipped to the range [lowest..highest]."""
    return max(lowest, min(x, highest))


def sigmoid_derivative(value):
    return value * (1 - value)


def sigmoid(x):
    """Return activation value of x with sigmoid function"""
    return 1 / (1 + math.exp(-x))


def relu_derivative(value):
    if value > 0:
        return 1
    else:
        return 0


def elu(x, alpha=0.01):
    if x > 0:
        return x
    else:
        return alpha * (math.exp(x) - 1)


def elu_derivative(value, alpha=0.01):
    if value > 0:
        return 1
    else:
        return alpha * math.exp(value)


def tanh(x):
    return np.tanh(x)


def tanh_derivative(value):
    return (1 - (value ** 2))


def leaky_relu(x, alpha=0.01):
    if x > 0:
        return x
    else:
        return alpha * x


def leaky_relu_derivative(value, alpha=0.01):
    if value > 0:
        return 1
    else:
        return alpha


def relu(x):
    return max(0, x)


def relu_derivative(value):
    if value > 0:
        return 1
    else:
        return 0


def step(x):
    """Return activation value of x with sign function"""
    return 1 if x >= 0 else 0


def gaussian(mean, st_dev, x):
    """Given the mean and standard deviation of a distribution, it returns the probability of x."""
    return 1 / (math.sqrt(2 * math.pi) * st_dev) * math.e ** (-0.5 * (float(x - mean) / st_dev) ** 2)


try:  # math.isclose was added in Python 3.5; but we might be in 3.4
    from math import isclose
except ImportError:
    def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
        """Return true if numbers a and b are close to each other."""
        return abs(a - b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

# ______________________________________________________________________________
# Grid Functions


orientations = EAST, NORTH, WEST, SOUTH = [(1, 0), (0, 1), (-1, 0), (0, -1)]
turns = LEFT, RIGHT = (+1, -1)


def turn_heading(heading, inc, headings=orientations):
    return headings[(headings.index(heading) + inc) % len(headings)]


def turn_right(heading):
    return turn_heading(heading, RIGHT)


def turn_left(heading):
    return turn_heading(heading, LEFT)


def distance(a, b):
    """The distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return math.hypot((xA - xB), (yA - yB))


def distance_squared(a, b):
    """The square of the distance between two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return (xA - xB) ** 2 + (yA - yB) ** 2


def vector_clip(vector, lowest, highest):
    """Return vector, except if any element is less than the corresponding
    value of lowest or more than the corresponding value of highest, clip to
    those values."""
    return type(vector)(map(clip, vector, lowest, highest))


# ______________________________________________________________________________
# Misc Functions

class injection():
    """Dependency injection of temporary values for global functions/classes/etc.
    E.g., `with injection(DataBase=MockDataBase): ...`"""

    def __init__(self, **kwds):
        self.new = kwds

    def __enter__(self):
        self.old = {v: globals()[v] for v in self.new}
        globals().update(self.new)

    def __exit__(self, type, value, traceback):
        globals().update(self.old)


def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn


def name(obj):
    """Try to find some reasonable name for the object."""
    return (getattr(obj, 'name', 0) or getattr(obj, '__name__', 0) or
            getattr(getattr(obj, '__class__', 0), '__name__', 0) or
            str(obj))


def isnumber(x):
    """Is x a number?"""
    return hasattr(x, '__int__')


def issequence(x):
    """Is x a sequence?"""
    return isinstance(x, collections.abc.Sequence)


def print_table(table, header=None, sep='   ', numfmt='{}'):
    """Print a list of lists as a table, so that columns line up nicely.
    header, if specified, will be printed as the first row.
    numfmt is the format for all numbers; you might want e.g. '{:.2f}'.
    (If you want different formats in different columns,
    don't use print_table.) sep is the separator between columns."""
    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]]

    if header:
        table.insert(0, header)

    table = [[numfmt.format(x) if isnumber(x) else x for x in row]
             for row in table]

    sizes = list(
        map(lambda seq: max(map(len, seq)),
            list(zip(*[map(str, row) for row in table]))))

    for row in table:
        print(sep.join(getattr(
            str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))


def open_data(name, mode='r'):
    aima_root = os.path.dirname(__file__)
    aima_file = os.path.join(aima_root, *['aima-data', name])

    return open(aima_file, mode=mode)


def failure_test(algorithm, tests):
    """Grades the given algorithm based on how many tests it passes.
    Most algorithms have arbitrary output on correct execution, which is difficult
    to check for correctness. On the other hand, a lot of algorithms output something
    particular on fail (for example, False, or None).
    tests is a list with each element in the form: (values, failure_output)."""
    from statistics import mean
    return mean(int(algorithm(x) != y) for x, y in tests)


# ______________________________________________________________________________
# Expressions

# See https://docs.python.org/3/reference/expressions.html#operator-precedence
# See https://docs.python.org/3/reference/datamodel.html#special-method-names

class Expr(object):
    """A mathematical expression with an operator and 0 or more arguments.
    op is a str like '+' or 'sin'; args are Expressions.
    Expr('x') or Symbol('x') creates a symbol (a nullary Expr).
    Expr('-', x) creates a unary; Expr('+', x, 1) creates a binary."""

    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

    # Operator overloads
    def __neg__(self):
        return Expr('-', self)

    def __pos__(self):
        return Expr('+', self)

    def __invert__(self):
        return Expr('~', self)

    def __add__(self, rhs):
        return Expr('+', self, rhs)

    def __sub__(self, rhs):
        return Expr('-', self, rhs)

    def __mul__(self, rhs):
        return Expr('*', self, rhs)

    def __pow__(self, rhs):
        return Expr('**', self, rhs)

    def __mod__(self, rhs):
        return Expr('%', self, rhs)

    def __and__(self, rhs):
        return Expr('&', self, rhs)

    def __xor__(self, rhs):
        return Expr('^', self, rhs)

    def __rshift__(self, rhs):
        return Expr('>>', self, rhs)

    def __lshift__(self, rhs):
        return Expr('<<', self, rhs)

    def __truediv__(self, rhs):
        return Expr('/', self, rhs)

    def __floordiv__(self, rhs):
        return Expr('//', self, rhs)

    def __matmul__(self, rhs):
        return Expr('@', self, rhs)

    def __or__(self, rhs):
        """Allow both P | Q, and P |'==>'| Q."""
        if isinstance(rhs, Expression):
            return Expr('|', self, rhs)
        else:
            return PartialExpr(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Expr('+', lhs, self)

    def __rsub__(self, lhs):
        return Expr('-', lhs, self)

    def __rmul__(self, lhs):
        return Expr('*', lhs, self)

    def __rdiv__(self, lhs):
        return Expr('/', lhs, self)

    def __rpow__(self, lhs):
        return Expr('**', lhs, self)

    def __rmod__(self, lhs):
        return Expr('%', lhs, self)

    def __rand__(self, lhs):
        return Expr('&', lhs, self)

    def __rxor__(self, lhs):
        return Expr('^', lhs, self)

    def __ror__(self, lhs):
        return Expr('|', lhs, self)

    def __rrshift__(self, lhs):
        return Expr('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Expr('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Expr('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Expr('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Expr('@', lhs, self)

    def __call__(self, *args):
        """Call: if 'f' is a Symbol, then f(0) == Expr('f', 0)."""
        if self.args:
            raise ValueError('can only do a call for a Symbol, not an Expr')
        else:
            return Expr(self.op, *args)

    # Equality and repr
    def __eq__(self, other):
        """x == y' evaluates to True or False; does not build an Expr."""
        return (isinstance(other, Expr)
                and self.op == other.op
                and self.args == other.args)

    def __lt__(self, other):
        return (isinstance(other, Expr)
                and str(self) < str(other))

    def __hash__(self):
        return hash(self.op) ^ hash(self.args)

    def __repr__(self):
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier():  # f(x) or f(x, y)
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1:  # -x or -(x + 1)
            return op + args[0]
        else:  # (x - y)
            opp = (' ' + op + ' ')
            return '(' + opp.join(args) + ')'


# An 'Expression' is either an Expr or a Number.
# Symbol is not an explicit type; it is any Expr with 0 args.


Number = (int, float, complex)
Expression = (Expr, Number)


def Symbol(name):
    """A Symbol is just an Expr with no args."""
    return Expr(name)


def symbols(names):
    """Return a tuple of Symbols; names is a comma/whitespace delimited str."""
    return tuple(Symbol(name) for name in names.replace(',', ' ').split())


def subexpressions(x):
    """Yield the subexpressions of an Expression (including x itself)."""
    yield x
    if isinstance(x, Expr):
        for arg in x.args:
            yield from subexpressions(arg)


def arity(expression):
    """The number of sub-expressions in this expression."""
    if isinstance(expression, Expr):
        return len(expression.args)
    else:  # expression is a number
        return 0


# For operators that are not defined in Python, we allow new InfixOps:


class PartialExpr:
    """Given 'P |'==>'| Q, first form PartialExpr('==>', P), then combine with Q."""

    def __init__(self, op, lhs):
        self.op, self.lhs = op, lhs

    def __or__(self, rhs):
        return Expr(self.op, self.lhs, rhs)

    def __repr__(self):
        return "PartialExpr('{}', {})".format(self.op, self.lhs)


def expr(x):
    """Shortcut to create an Expression. x is a str in which:
    - identifiers are automatically defined as Symbols.
    - ==> is treated as an infix |'==>'|, as are <== and <=>.
    If x is already an Expression, it is returned unchanged. Example:
    >>> expr('P & Q ==> Q')
    ((P & Q) ==> Q)
    """
    if isinstance(x, str):
        return eval(expr_handle_infix_ops(x), defaultkeydict(Symbol))
    else:
        return x


infix_ops = '==> <== <=>'.split()


def expr_handle_infix_ops(x):
    """Given a str, return a new str with ==> replaced by |'==>'|, etc.
    >>> expr_handle_infix_ops('P ==> Q')
    "P |'==>'| Q"
    """
    for op in infix_ops:
        x = x.replace(op, '|' + repr(op) + '|')
    return x


class defaultkeydict(collections.defaultdict):
    """Like defaultdict, but the default_factory is a function of the key.
    >>> d = defaultkeydict(len); d['four']
    4
    """

    def __missing__(self, key):
        self[key] = result = self.default_factory(key)
        return result


class hashabledict(dict):
    """Allows hashing by representing a dictionary as tuple of key:value pairs
       May cause problems as the hash value may change during runtime
    """

    def __hash__(self):
        return 1


# ______________________________________________________________________________
# Queues: Stack, FIFOQueue, PriorityQueue
# Stack and FIFOQueue are implemented as list and collection.deque
# PriorityQueue is implemented here


class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []

        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)


# ______________________________________________________________________________
# Monte Carlo tree node and ucb function
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states"""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    return (float('inf') if n.N == 0 else
            n.U / n.N + C * math.sqrt(math.log(n.parent.N) / n.N))


# ______________________________________________________________________________
# Useful Shorthands


class Bool(int):
    """Just like `bool`, except values display as 'T' and 'F' instead of 'True' and 'False'"""
    __str__ = __repr__ = lambda self: 'T' if self else 'F'


T = Bool(True)
F = Bool(False)


In [2]:
# these functions and classes are taken from https://github.com/aimacode/aima-python
# which is the github for the "Artifical Intelligence, A Modern Approach"

import bisect
import math
import random
import sys
from collections import deque




infinity = float('inf')

# ______________________________________________________________________________


class Problem(object):
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError


# ______________________________________________________________________________


class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                         problem.path_cost(self.path_cost, self.state,
                                           action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)
    

    
# -----------------------Problem Subclasses---------------------------------

def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    count = 0
    while frontier:
        node = frontier.pop()
        count += 1
        if problem.goal_test(node.state):
            print(f'Total explored nodes = {count}\n')
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return 



def astar_search(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n))

In [3]:
#This is the 3x3 vaccum world problem that I coded. It uses the classes and functions defined above to work.

class Vaccuum_World(Problem):
    """
    Subclass of problem for the vaccuum world problem defined in homework. 
    The world is a 3x3 board of squares described by (x, y) with location (0, 0)
    being the top left square. each square either has dirt on it or not, represented
    by a 9-tuple of boolean values, 1 for dirty, 0 for clean. States are represented
    by a 2-tuple containing the current (x, y) location of the vaccuum, and the 
    dirt distribution of the tiles. 
    Example state: ((0,2), (0,0,1,1,0,0,0,0,1))
    The default goal is when none of the squares have dirt on them, represented by a 
    9-tuple of 0's
    """
    
    def __init__(self, initial, goal=(0,0,0,0,0,0,0,0,0)):
        '''Define the goal state an initialize the problem'''
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self, state):
        '''
        Returns the actions that can be excecuted in a given state.
        The result is a list containing 3 to 5 of the 5 possible actions
        '''
        
        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT', 'SUCK']
        vaccuum_location = list(state[0])
        dirt_distribution = list(state[1])
        
        #current tile is the index of the location in the dirt_distribution
        current_tile = vaccuum_location[0] + (vaccuum_location[1] * 3)
        
        
        if vaccuum_location[0] == 0:
            possible_actions.remove('LEFT')
        if vaccuum_location[1] == 0:
            possible_actions.remove('UP')
        if vaccuum_location[0] == 2:
            possible_actions.remove('RIGHT')
        if vaccuum_location[1] == 2:
            possible_actions.remove('DOWN')
        if dirt_distribution[current_tile] == 0:
            possible_actions.remove('SUCK')
            
        return possible_actions
    
    def result(self, state, action):
        '''Given a state and an action, return a new state that is the result
        of that action.'''
        
        vaccuum_location = list(state[0])
        dirt_distribution = list(state[1])
        
        #current tile is the index of the location in the dirt_distribution
        current_tile = vaccuum_location[0] + (vaccuum_location[1] * 3)
        
        #delta is the (x, y) movement as a result of the action
        delta = {'UP': (0,-1), 'DOWN': (0, 1),
                 'LEFT': (-1, 0), 'RIGHT': (1, 0)}
        
        if action == 'SUCK':
            dirt_distribution[current_tile] = 0
            new_location = tuple(vaccuum_location)
        else:
            new_location = (vaccuum_location[0] + delta[action][0], vaccuum_location[1]+delta[action][1])
        new_state = (new_location, tuple(dirt_distribution))
        
        return new_state
    
    def goal_test(self, state):
        '''given a state, return true if it is the goal state, False if not'''
        
        return state[1] == self.goal
    
    def path_cost(self, c, state1, action, state2):
        ''' 
        If the vaccuum moves, the path cost is 1 + 2(# of dirty squares)
        If the vaccuum sucks, the path cost is 1 + 2(# of dirty squares - 1)
        '''
        state = list(state2)
        dirt_distribution = list(state[1])
        
        squares_dirty = dirt_distribution.count(1)        
        
        
        if action == 'SUCK':
            cost = 1 + 2*(squares_dirty)
        else:
            cost = 1 + 2*(squares_dirty)
        
        return cost
    
    def h(self, node):
        '''
        Return the heuristic value for the given state. Default is:
        
        h(n)=2*w-1 + d   
        
        where w = # of dirty squares at the state for node n,
        and d = the shortest distance between the current square and a dirty square
        '''
        

        node_state = list(node.state)
        vaccuum_location = list(node_state[0])
        dirt_distribution = list(node_state[1])
        dirty_indexes = []
        distances = []
        num_dirty = 0
        
        # loops through the dirt_distribution and counts how many are dirty
        # and adds their indexes to a list for future calculation
        for index, ele in enumerate(dirt_distribution, 0):
            if ele == 1:
                dirty_indexes.append(index)
                num_dirty += 1
         
        # finds the distance between each dirty square and the vaccuum square and adds it to a list
        d = infinity
        for item in dirty_indexes:
            x = item - (int(item/3)*3)
            y = int(item/3)
            dist = abs(x - vaccuum_location[0]) + abs(y - vaccuum_location[1])
            if dist < d:
                d = dist
        
        # calculates d by finding the distance from the vaccuum to the nearest dirty square
        
        h = (2*num_dirty-1) + d
        
        return h

                
        
            
        
        
        

In [4]:
# Run the program for the 3 by 3 grid world with top 3 squares dirty, and agent in the center square
# (second row and second column) as the initial state. 


initial_location = (1,1)

# The dirt distribution is represnted by a set of booleans, 1 for dirty, 0 for clean.

initial_dirt_distribution = (1,1,1,0,0,0,0,0,0)

#States are represnted by a 2-tuple of location and dirt distribution
initial_state = (initial_location, initial_dirt_distribution)





# Heuristics for the problem are defined here

def h2(node):
    '''
    This heuristic is defined such that h2 is dominated by the default heuristic of Vaccuum_World

    h2(n)=2*w-1

    where w = # of dirty squares at the state for node n,
    and d = the shortest distance between the current square and a dirty square
    '''


    node_state = list(node.state)
    vaccuum_location = list(node_state[0])
    dirt_distribution = list(node_state[1])
    dirty_indexes = []
    distances = []
    num_dirty = 0

    # loops through the dirt_distribution and counts how many are dirty
    # and adds their indexes to a list for future calculation
    for index, ele in enumerate(dirt_distribution, 0):
        if ele == 1:
            dirty_indexes.append(index)
            num_dirty += 1


    # calculates d by finding the distance from the vaccuum to the nearest dirty square

    h2 = (2*num_dirty-1) 

    return h2

In [5]:
# This is the Vaccuum-World run with its default heuristic. The solution and total nodes explored, 
# along with the steps taken and total cost of those steps is presented here.


puzzle = Vaccuum_World(initial_state)
print('The solution to the puzzle is : \n')
print(astar_search(puzzle).solution())
print('\n\n')
print('The sequence of states and costs to reach that state is :\n')
cost = 0
for x in astar_search(puzzle).path()[:]:
    cost += x.path_cost
    print(f'The action {x.action} cost {x.path_cost} \nand brings us to state {x.state} \nfor a total cost of {cost}\n')
    


The solution to the puzzle is : 

Total explored nodes = 64

['UP', 'SUCK', 'RIGHT', 'SUCK', 'LEFT', 'LEFT', 'SUCK']



The sequence of states and costs to reach that state is :

Total explored nodes = 64

The action None cost 0 
and brings us to state ((1, 1), (1, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 0

The action UP cost 7 
and brings us to state ((1, 0), (1, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 7

The action SUCK cost 5 
and brings us to state ((1, 0), (1, 0, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 12

The action RIGHT cost 5 
and brings us to state ((2, 0), (1, 0, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 17

The action SUCK cost 3 
and brings us to state ((2, 0), (1, 0, 0, 0, 0, 0, 0, 0, 0)) 
for a total cost of 20

The action LEFT cost 3 
and brings us to state ((1, 0), (1, 0, 0, 0, 0, 0, 0, 0, 0)) 
for a total cost of 23

The action LEFT cost 3 
and brings us to state ((0, 0), (1, 0, 0, 0, 0, 0, 0, 0, 0)) 
for a total cost of 26

The action SUCK cost 1 
and

In [6]:
# This is the Vaccuum-World run with its h2 heuristic. The solution and total nodes explored,
# along with the steps taken and total cost of those steps is presented here.


puzzle = Vaccuum_World(initial_state)
print('The solution to the puzzle is : ')
print(astar_search(puzzle, h2).solution())
print('\n\n')
print('The sequence of states and costs to reach that state is :\n')
cost = 0
for x in astar_search(puzzle, h2).path()[:]:
    cost += x.path_cost
    print(f'The action {x.action} cost {x.path_cost} \nand brings us to state {x.state} \nfor a total cost of {cost}\n')
    

The solution to the puzzle is : 
Total explored nodes = 15

['LEFT', 'UP', 'SUCK', 'RIGHT', 'SUCK', 'RIGHT', 'SUCK']



The sequence of states and costs to reach that state is :

Total explored nodes = 15

The action None cost 0 
and brings us to state ((1, 1), (1, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 0

The action LEFT cost 7 
and brings us to state ((0, 1), (1, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 7

The action UP cost 7 
and brings us to state ((0, 0), (1, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 14

The action SUCK cost 5 
and brings us to state ((0, 0), (0, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 19

The action RIGHT cost 5 
and brings us to state ((1, 0), (0, 1, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 24

The action SUCK cost 3 
and brings us to state ((1, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 27

The action RIGHT cost 3 
and brings us to state ((2, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0)) 
for a total cost of 30

The action SUCK cost 1 
an